## PyMilvus dev

In [1]:
from semantic_search.utils import load_metadata

df, ref_df = load_metadata(
    '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/metadata3',
    filter_good_papers=True,
    filter_good_references=True
)
ref_df.rename(columns={'oaid': 'id', 'abstract': 'text'}, inplace=True)

In [13]:
from importlib import reload
from semantic_search.store import models, milvus_store
reload(models)
reload(milvus_store)
from semantic_search.store.models import LocalEmbeddingModel
from semantic_search.store.milvus_store import MilvusDocumentStore

model = LocalEmbeddingModel()

ds = MilvusDocumentStore(
    model=model, 
    db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/milvus-dev1',
    store_documents=True
)


Using device: cuda


FileExistsError: [Errno 17] File exists: '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/milvus-dev1.db'

In [11]:
ds.create_index_from_df(ref_df.iloc[:100], overwrite=True)

OSError: Cannot save file into a non-existent directory: '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/milvus-dev1.db'

In [5]:
ds.db_dir, ds._client_db_dir

('/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/milvus-dev1.db',
 '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/milvus-dev1.db')

In [4]:
ds.search('Deep residual networks', top_k=100)

[{'rank': 1,
  'text': 'deeper neural networks are more difficult to train. we present a residual learning framework to ease the training of networks that are substantially deeper than those used previously. we explicitly reformulate the layers as learning residual functions with reference to the layer inputs, instead of learning unreferenced functions. we provide comprehensive empirical evidence showing that these residual networks are easier to optimize, and can gain accuracy from considerably increased depth. on the imagenet dataset we evaluate residual nets with a depth of up to 152 layers - 8× deeper than vgg nets [ 40 ] but still having lower complexity. an ensemble of these residual nets achieves 3. 57 % error on the imagenet test set. this result won the 1st place on the ilsvrc 2015 classification task. we also present analysis on cifar - 10 with 100 and 1000 layers. the depth of representations is of central importance for many visual recognition tasks. solely due to our extre

## Embedding dev (tokenisation, embedding and pooling)

In [6]:
from importlib import reload
from semantic_search import store
reload(store)
from semantic_search.store.models import LocalEmbeddingModel

model = LocalEmbeddingModel()

Using device: cpu


In [5]:
model.embedding_dim

384

In [234]:
import torch

tmp_input = "Using pooling, it generates from a variable sized sentence a fixed sized sentence embedding. This layer also allows to use the CLS token if it is returned by the underlying word embedding model. You can concatenate multiple poolings together."

inputs = [tmp_input*10, tmp_input*3, tmp_input, tmp_input*30]

_, all_chunks_encoded = model.chunk_and_encode(inputs, progress_bar=True)

# Flatten encoded
tmp = {}
for single_text_encoded in all_chunks_encoded:
    for k, v in single_text_encoded.items():
        if k not in tmp:
            tmp[k] = []
        tmp[k].append(v)
encoded_flattened = {k: torch.cat(v) for k, v in tmp.items()}

# Get embeddings for all chunks
print(f"Generating embeddings for {len(list(encoded_flattened.values())[0])} chunks...")
embeddings = model.get_embeddings(encoded_flattened, progress_bar=True)

Chunking and encoding: 100%|██████████| 4/4 [00:00<00:00, 585.61it/s]


Generating embeddings for 8 chunks...


Generating embeddings: 100%|██████████| 1/1 [00:02<00:00,  2.76s/it]


In [231]:
from torch.nn import functional as F

_, encoded = model.chunk_and_encode(tmp_input*100)
emb = model.get_embeddings(encoded)


encoded_input = model.tokenizer(tmp_input*100, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model.model(**encoded_input)

# Perform pooling
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)



In [232]:
emb[0] == sentence_embeddings[0]

tensor([True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, True, True, True, True, True, True, True,
        True, True, True, True, True, Tr

In [1]:
from semantic_search.utils import load_metadata

df, ref_df = load_metadata(
    '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/raw-data/metadata3',
    filter_good_papers=True,
    filter_good_references=True
)

In [3]:
ref_df['']

,oaid,doi,ref_via,title,abstract,type,topic,domain,field,subfield
0,W2194775991,10.1109/cvpr.2016.90,openalex_id,Deep Residual Learning for Image Recognition,Deeper neural networks are more difficult to t...,article,Advanced Neural Network Applications,Physical Sciences,Computer Science,Computer Vision and Pattern Recognition


In [2]:
# Running below in REPL works

import os

# Set PyTerrier home directory
os.environ['PYTERRIER_HOME'] = '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev'
os.environ['TMPDIR'] = '/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev/tmp'

import pyterrier as pt
import pandas as pd

if not pt.started(): pt.init(home_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev')

# import your documents into an index
# Create a pandas DataFrame with the documents

doc_texts = ref_df.abstract

docs_df = pd.DataFrame({
    'docno': [str(i) for i in range(len(doc_texts))],
    'text': doc_texts
})

# Use IterDictIndexer instead of TerrierIndexer
indexer = pt.IterDictIndexer('/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev/terrier_index')
index_ref = indexer.index(docs_df.to_dict('records'))

bm25 = pt.BatchRetrieve(index_ref, wmodel="BM25")
results = bm25.transform(pt.Utils.prepare_qrels(pd.DataFrame({'qid':['q1'], 'query':['your keywords here']})))
print(results[['qid', 'docno', 'score']])

/scratch/tmp.31851532.lcarretero/ipykernel_946913/2934055006.py:11: DeprecationWarning: Call to deprecated function (or staticmethod) started. (use pt.java.started() instead) -- Deprecated since version 0.11.0.
  if not pt.started(): pt.init(home_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev')


terrier-assemblies 5.11 jar-with-dependencies not found, downloading to /cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/src/semantic_search/dev...


PermissionError: [Errno 13] Permission denied: '/terrier-assemblies-5.11-jar-with-dependencies.jar4pglkysh.tmp'

## Dev for new cleaner store setup

In [1]:
from importlib import reload
from semantic_search.store import store
reload(store)
from semantic_search.store.store import FAISSDocumentStore
from tqdm import tqdm

Exception: Unable to find javac

In [2]:
ds = FAISSDocumentStore(db_dir='/cluster/home/lcarretero/workspace/dsl/dsl-research-assistant/db/hybrid-dev3')
assert ds.load_store()

2025-05-13 12:32:08.412831: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-13 12:32:09.511891: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-05-13 12:32:11.497976: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Using device: cpu
Loaded index with 41220 vectors


In [7]:
ds.search('Deep residual networks', top_k=10, retrieval_method='hybrid')

[{'rank': 1,
  'id': 'W2964137095',
  'doi': '10.5244/c.30.87',
  'ref_via': 'openalex_id',
  'title': 'Wide Residual Networks',
  'text': 'Deep residual networks were shown to be able to scale up to thousands of layers and still have improving performance. However, each fraction of a percent of improved accuracy costs nearly doubling the number of layers, and so training very deep residual networks has a problem of diminishing feature reuse, which makes these networks very slow to train. To tackle these problems, in this paper we conduct a detailed experimental study on the architecture of ResNet blocks, based on which we propose a novel architecture where we decrease depth and increase width of residual networks. We call the resulting network structures wide residual networks (WRNs) and show that these are far superior over their commonly used thin and very deep counterparts. For example, we demonstrate that even a simple 16-layer-deep wide residual network outperforms in accuracy an